In [1]:


sql_script = """ 
-- completedorder table
CREATE TABLE completedorder (
    order_id INTEGER PRIMARY KEY,
    account_id TEXT,
    bank_to TEXT,
    account_to INTEGER,
    amount REAL,
    k_symbol TEXT,
    FOREIGN KEY (account_id) REFERENCES completedacct(account_id)
);

-- CRM Events table
CREATE TABLE CRM_Events (
    Date_received TEXT,
    Product TEXT,
    Sub_product TEXT,
    Issue TEXT,
    Sub_issue REAL,
    Consumer_complaint_narrative TEXT,
    Tags TEXT,
    Consumer_consent_provided TEXT,
    Submitted_via TEXT,
    Date_sent_to_company TEXT,
    Company_response_to_consumer TEXT,
    Timely_response TEXT,
    Consumer_disputed TEXT,
    Complaint_ID TEXT PRIMARY KEY,
    Client_ID TEXT,
    FOREIGN KEY (Client_ID) REFERENCES completedclient(client_id)
);

-- completeddisposition table
CREATE TABLE completeddisposition (
    disp_id TEXT PRIMARY KEY,
    client_id TEXT,
    account_id TEXT,
    type TEXT,
    FOREIGN KEY (client_id) REFERENCES completedclient(client_id),
    FOREIGN KEY (account_id) REFERENCES completedacct(account_id)
);

-- completedcard table
CREATE TABLE completedcard (
    card_id TEXT PRIMARY KEY,
    disp_id TEXT,
    type TEXT,
    year INTEGER,
    month INTEGER,
    day INTEGER,
    fulldate TEXT,
    FOREIGN KEY (disp_id) REFERENCES completeddisposition(disp_id)
);

-- LuxuryLoanPortfolio table
CREATE TABLE LuxuryLoanPortfolio (
    loan_id TEXT PRIMARY KEY,
    funded_amount REAL,
    funded_date TEXT,
    duration_years INTEGER,
    duration_months INTEGER,
    treasury_index_date_funded REAL,
    interest_rate_percent REAL,
    interest_rate REAL,
    payments REAL,
    total_past_payments INTEGER,
    loan_balance REAL,
    property_value REAL,
    purpose TEXT,
    firstname TEXT,
    middlename TEXT,
    lastname TEXT,
    social TEXT,
    phone TEXT,
    title TEXT,
    employment_length INTEGER,
    building_class_category TEXT,
    tax_class_at_present TEXT,
    building_class_at_present TEXT,
    address_1 TEXT,
    address_2 TEXT,
    zip_code INTEGER,
    city TEXT,
    state TEXT,
    total_units INTEGER,
    land_square_feet TEXT,
    gross_square_feet TEXT,
    tax_class_at_time_of_sale INTEGER
);

-- CRM Call Center Logs table
CREATE TABLE CRM_Call_Center_Logs (
    Date_received TEXT,
    Complaint_ID TEXT,
    rand_client TEXT,
    phonefinal TEXT,
    vru_line TEXT,
    call_id REAL,
    priority REAL,
    type TEXT,
    outcome TEXT,
    server TEXT,
    ser_start TEXT,
    ser_exit TEXT,
    ser_time TEXT,
    FOREIGN KEY (Complaint_ID) REFERENCES CRM_Events(Complaint_ID)
);

-- completedloan table
CREATE TABLE completedloan (
    loan_id TEXT PRIMARY KEY,
    account_id TEXT,
    amount INTEGER,
    duration INTEGER,
    payments INTEGER,
    status TEXT,
    year INTEGER,
    month INTEGER,
    day INTEGER,
    fulldate TEXT,
    location INTEGER,
    purpose TEXT,
    FOREIGN KEY (account_id) REFERENCES completedacct(account_id)
);

-- completeddistrict table
CREATE TABLE completeddistrict (
    district_id INTEGER PRIMARY KEY,
    city TEXT,
    state_name TEXT,
    state_abbrev TEXT,
    region TEXT,
    division TEXT
);

-- completedtrans table
CREATE TABLE completedtrans (
    Unnamed_0 INTEGER,
    trans_id TEXT PRIMARY KEY,
    account_id TEXT,
    type TEXT,
    operation TEXT,
    amount REAL,
    balance REAL,
    k_symbol TEXT,
    bank TEXT,
    account REAL,
    year INTEGER,
    month INTEGER,
    day INTEGER,
    fulldate TEXT,
    fulltime TEXT,
    fulldatewithtime TEXT,
    FOREIGN KEY (account_id) REFERENCES completedacct(account_id)
);

-- CRM Reviews table
CREATE TABLE CRM_Reviews (
    Date TEXT,
    Stars INTEGER,
    Reviews TEXT,
    Product TEXT,
    district_id INTEGER,
    FOREIGN KEY (district_id) REFERENCES completeddistrict(district_id)
);

-- completedacct table
CREATE TABLE completedacct (
    account_id TEXT PRIMARY KEY,
    district_id INTEGER,
    frequency TEXT,
    parseddate TEXT,
    year INTEGER,
    month INTEGER,
    day INTEGER,
    FOREIGN KEY (district_id) REFERENCES completeddistrict(district_id)
);

-- completedclient table
CREATE TABLE completedclient (
    client_id TEXT PRIMARY KEY,
    sex TEXT,
    fulldate TEXT,
    day INTEGER,
    month INTEGER,
    year INTEGER,
    age INTEGER,
    social TEXT,
    first TEXT,
    middle TEXT,
    last TEXT,
    phone TEXT,
    email TEXT,
    address_1 TEXT,
    address_2 TEXT,
    city TEXT,
    state TEXT,
    zipcode INTEGER,
    district_id INTEGER,
    FOREIGN KEY (district_id) REFERENCES completeddistrict(district_id)
);
 """

In [2]:
import sqlite3
new_db = "crm_refined.sqlite3"
conn = sqlite3.connect(new_db)
c = conn.cursor()
c.executescript(sql_script)
conn.commit()
conn.close()
print("Database created successfully")

Database created successfully


In [12]:
import sqlite3

def copy_table_data(old_cursor, new_cursor, table_name, columns):
    old_cursor.execute(f"SELECT * FROM {table_name}")
    rows = old_cursor.fetchall()
    placeholders = ', '.join('?' * len(columns))
    new_cursor.executemany(f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({placeholders})", rows)

def main():
    old_db = '../../databases/crm1.db'  # Path to your old database file
    new_db = 'crm_refined.sqlite3'  # Path to your new database file

    # Connect to the old and new databases
    with sqlite3.connect(old_db) as old_conn, sqlite3.connect(new_db) as new_conn:
        old_cursor = old_conn.cursor()
        new_cursor = new_conn.cursor()

        # Temporarily disable foreign key constraints in the new database
        new_cursor.execute("PRAGMA foreign_keys = OFF")

        # Define the tables and their columns to be copied
        tables_to_copy = {
            'completedorder': ['order_id', 'account_id', 'bank_to', 'account_to', 'amount', 'k_symbol'],
            'CRM_Events': ['Date_received', 'Product', 'Sub_product', 'Issue', 'Sub_issue', 'Consumer_complaint_narrative', 'Tags', 'Consumer_consent_provided', 'Submitted_via', 'Date_sent_to_company', 'Company_response_to_consumer', 'Timely_response', 'Consumer_disputed', 'Complaint_ID', 'Client_ID'],
            'completeddisposition': ['disp_id', 'client_id', 'account_id', 'type'],
            'completedcard': ['card_id', 'disp_id', 'type', 'year', 'month', 'day', 'fulldate'],
            'LuxuryLoanPortfolio': ['loan_id', 'funded_amount', 'funded_date', 'duration_years', 'duration_months', 'treasury_index_date_funded', 'interest_rate_percent', 'interest_rate', 'payments', 'total_past_payments', 'loan_balance', 'property_value', 'purpose', 'firstname', 'middlename', 'lastname', 'social', 'phone', 'title', 'employment_length', 'building_class_category', 'tax_class_at_present', 'building_class_at_present', 'address_1', 'address_2', 'zip_code', 'city', 'state', 'total_units', 'land_square_feet', 'gross_square_feet', 'tax_class_at_time_of_sale'],
            'CRM_Call_Center_Logs': ['Date_received', 'Complaint_ID', 'rand_client', 'phonefinal', 'vru_line', 'call_id', 'priority', 'type', 'outcome', 'server', 'ser_start', 'ser_exit', 'ser_time'],
            'completedloan': ['loan_id', 'account_id', 'amount', 'duration', 'payments', 'status', 'year', 'month', 'day', 'fulldate', 'location', 'purpose'],
            'completeddistrict': ['district_id', 'city', 'state_name', 'state_abbrev', 'region', 'division'],
            'completedtrans': ['Unnamed_0', 'trans_id', 'account_id', 'type', 'operation', 'amount', 'balance', 'k_symbol', 'bank', 'account', 'year', 'month', 'day', 'fulldate', 'fulltime', 'fulldatewithtime'],
            'CRM_Reviews': ['Date', 'Stars', 'Reviews', 'Product', 'district_id'],
            'completedacct': ['account_id', 'district_id', 'frequency', 'parseddate', 'year', 'month', 'day'],
            'completedclient': ['client_id', 'sex', 'fulldate', 'day', 'month', 'year', 'age', 'social', 'first', 'middle', 'last', 'phone', 'email', 'address_1', 'address_2', 'city', 'state', 'zipcode', 'district_id']
        }


        # Copy data for each table
        for table, columns in tables_to_copy.items():
            print(f"Copying data for table: {table}")
            copy_table_data(old_cursor, new_cursor, table, columns)

        # Re-enable foreign key constraints
        new_cursor.execute("PRAGMA foreign_keys = ON")

        # Commit the changes
        new_conn.commit()

        print("Data copying completed.")

if __name__ == "__main__":
    main()


Copying data for table: completedorder
Copying data for table: CRM_Events
Copying data for table: completeddisposition
Copying data for table: completedcard
Copying data for table: LuxuryLoanPortfolio
Copying data for table: CRM_Call_Center_Logs
Copying data for table: completedloan
Copying data for table: completeddistrict
Copying data for table: completedtrans
Copying data for table: CRM_Reviews
Copying data for table: completedacct
Copying data for table: completedclient
Data copying completed.


In [9]:
import sqlite3

def rename_table(db_file, old_table_name, new_table_name):
    try:
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()
        cursor.execute(f"ALTER TABLE `{old_table_name}` RENAME TO `{new_table_name}`")
        conn.commit()
        print(f"Table '{old_table_name}' has been renamed to '{new_table_name}'")
    except sqlite3.Error as error:
        print(f"Error occurred: {error}")
    finally:
        if conn:
            conn.close()

# Usage
db_path = '../../databases/crm1.db'  # Replace with the path to your database

# Renaming tables
rename_table(db_path, 'CRM Events', 'CRM_Events'.replace(" ", "_"))
rename_table(db_path, 'CRM Call Center Logs', 'CRM_Call_Center_Logs'.replace(" ", "_"))
rename_table(db_path, 'CRM Reviews', 'CRM_Reviews'.replace(" ", "_"))




Table 'CRM Events' has been renamed to 'CRM_Events'
Table 'CRM Call Center Logs' has been renamed to 'CRM_Call_Center_Logs'
Table 'CRM Reviews' has been renamed to 'CRM_Reviews'
